# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sun Sep 19 02:49:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 452.56       Driver Version: 452.56       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:2D:00.0 Off |                  N/A |
| N/A   50C    P8    N/A /  N/A |    119MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2021년 Hyundai Data**

In [5]:
lexicon_2021 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2021_hyundai.csv')

lexicon_2021.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,현대차,5380,매일경제,2021010112,[풀영상] 2021 영동대로 밤하늘을 자동차가 날았다,http://news.mk.co.kr/newsRead.php?no=1330&year...,\n\n\n\n▶ 여기를 누르시면 크게 보실 수 있습니다\n 현대자동차가 서울 강...,2021-01-04,12,194000,211000,193500,207500,5651695,0.080729,1,1,여기 크게 자동차 서울 강남 영동대로 촬영 신축년 환영 드론 영상 공개 현대차 지난...


In [6]:
lexicon_2021[lexicon_2021['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2021.dropna(axis=0, inplace=True)
lexicon_2021 = lexicon_2021.reset_index(drop=True)
lexicon_2021.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
neg_lexicon_2021 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_neg_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_neg와의 Cosine Similarity 계산
for x in range(len(lexicon_2021['Tokenization'])):
    news_num = x+1
    for y in range(len(negative)):
        for z in range(len(list(set(lexicon_2021['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(negative[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(negative[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2021['Tokenization'][x].split())):
                    if lexicon_2021['Tokenization'][x].split()[w] == list(set(lexicon_2021['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2021['date'][x],
                    'news_num': news_num,
                    'KOSELF_neg_word': negative[y],
                    'news_word': list(set(lexicon_2021['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(negative[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                neg_lexicon_2021 = neg_lexicon_2021.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2021['date'][x], negative[y], list(set(lexicon_2021['Tokenization'][x].split()))[z], ko_model.wv.similarity(negative[y], list(set(lexicon_2021['Tokenization'][x].split()))[z])))

***2021-01-04 Cosine Similarity between <과적> & <화물> : 0.5248162746429443
***2021-01-04 Cosine Similarity between <과적> & <차량> : 0.5693626403808594
***2021-01-04 Cosine Similarity between <의심> & <의심> : 0.9999998807907104
***2021-01-04 Cosine Similarity between <과적> & <차량> : 0.5693626403808594
***2021-01-04 Cosine Similarity between <둔화> & <증가> : 0.5041859149932861
***2021-01-04 Cosine Similarity between <침체> & <위기> : 0.5072819590568542
***2021-01-04 Cosine Similarity between <하락> & <증가> : 0.553473711013794
***2021-01-04 Cosine Similarity between <둔화> & <증가> : 0.5041859149932861
***2021-01-04 Cosine Similarity between <둔화> & <감소> : 0.5349085330963135
***2021-01-04 Cosine Similarity between <배상> & <보상> : 0.5126209259033203
***2021-01-04 Cosine Similarity between <약화> & <감소> : 0.551184892654419
***2021-01-04 Cosine Similarity between <하락> & <증가> : 0.553473711013794
***2021-01-04 Cosine Similarity between <하락> & <감소> : 0.575989305973053
***2021-01-04 Cosine Similarity between <둔화> & <반등> : 0

In [11]:
neg_lexicon_2021

,date,news_num,KOSELF_neg_word,news_word,cosine_similarity,frequency
0,2021-01-04,3,과적,화물,0.524816,1
1,2021-01-04,5,과적,차량,0.569363,1
2,2021-01-04,5,의심,의심,1.000000,1
3,2021-01-04,6,과적,차량,0.569363,3
4,2021-01-04,6,둔화,증가,0.504186,2
...,...,...,...,...,...,...
11404,2021-09-16,3381,침체,위축,0.550398,1
11405,2021-09-16,3381,하락,반등,0.575454,4
11406,2021-09-16,3381,하락,상승,0.712648,23
11407,2021-09-16,3381,하락,보합,0.617605,2


In [12]:
neg_lexicon_2021.to_csv('../../../../Code/Data/Test/Stock-Year/neg_hyundai_2021.csv', index=False)

In [13]:
a = list(set(list(neg_lexicon_2021['news_word'])))

a

['실린더헤드',
 '절상',
 '악화',
 '강화',
 '증차',
 '호전',
 '차량',
 '난폭',
 '반등',
 '탑차',
 '실패',
 '손해배상',
 '상승',
 '하락',
 '항행',
 '악재',
 '좌절',
 '경기',
 '위축',
 '불황',
 '위기',
 '금값',
 '질책',
 '등락',
 '엠앤티',
 '부도',
 '스타벤처스',
 '딜러숍',
 '화물',
 '보상',
 '주춤',
 '논란',
 '학연',
 '급락',
 '보합',
 '선령',
 '손괴',
 '플러스',
 '혼돈',
 '급감',
 '스캔들',
 '증가',
 '급등',
 '참패',
 '약세',
 '호황',
 '우발',
 '누유',
 '역풍',
 '견조',
 '연루',
 '약화',
 '폭락',
 '둔화',
 '추징',
 '업황',
 '스위블링',
 '선박',
 '끝내',
 '성공',
 '결항',
 '강점',
 '운항',
 '의심',
 '경색',
 '불안',
 '특혜',
 '비리',
 '유발',
 '감소',
 '폭등',
 '침체',
 '격벽',
 '저하',
 '급유',
 '파산보호',
 '합선',
 '당혹',
 '폐업',
 '과속']

In [14]:
b = list(set(list(neg_lexicon_2021[neg_lexicon_2021['cosine_similarity']>=0.7]['news_word'])))

b

['침체', '스캔들', '약화', '의심', '실패', '성공', '상승']

In [15]:
c = list(set(list(neg_lexicon_2021[neg_lexicon_2021['cosine_similarity']>=0.65]['news_word'])))

c

['침체', '급락', '스캔들', '약화', '의심', '실패', '성공', '상승']